In [65]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Question Answering Analysis

## Content

### Load datasets

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pylab import rcParams


%matplotlib inline

In [3]:
sns.set(style="ticks")
sns.set_style("whitegrid")
rcParams['figure.dpi'] = 350
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'
rcParams['figure.figsize'] = 15,10
rcParams['font.size'] = 20
rcParams['axes.labelsize'] = 'large'
rcParams['xtick.labelsize'] = 20
rcParams['ytick.labelsize'] = 20

In [4]:
import os
import sys
import gzip
import json
import nltk
from nltk import clean_html

In [5]:
from textblob import TextBlob, Word

In [6]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


In [7]:
application = getDF('/data/QA/Appliances.json.gz')

In [8]:
application.head()

,questionType,asin,answerTime,unixTime,question,answerType,answer
0,yes/no,B00004U9JP,"Jun 27, 2014",1.403852e+09,I have a 9 year old Badger 1 that needs replac...,?,I replaced my old one with this without a hitch.
1,open-ended,B00004U9JP,"Apr 28, 2014",1.398668e+09,model number,NaN,This may help InSinkErator Model BADGER-1: Bad...
2,yes/no,B00004U9JP,"Aug 25, 2014",1.408950e+09,can I replace Badger 1 1/3 with a Badger 5 1/2...,?,Plumbing connections will vary with different ...
3,yes/no,B00004U9JP,"Nov 3, 2014",1.415002e+09,Does this come with power cord and dishwasher ...,?,It does not come with a power cord. It does co...
4,open-ended,B00004U9JP,"Jun 21, 2014",1.403334e+09,loud noise inside when turned on. sounds like ...,NaN,Check if you dropped something inside.Usually ...


# 2 TextBlob

In [9]:
import nltk
from textblob import TextBlob, Word

In [10]:
blob = TextBlob(application['question'][0])

blob

TextBlob("I have a 9 year old Badger 1 that needs replacing, will this Badger 1 install just like the original one?")

In [11]:
blob.noun_phrases
#??????

WordList(['badger', 'badger'])

In [12]:
blob.correct()
# 改错了

TextBlob("I have a 9 year old Danger 1 that needs replacing, will this Danger 1 install just like the original one?")

## Sentiment Analysis with TextBlob

#### #

In [13]:
from textblob.sentiments import NaiveBayesAnalyzer, PatternAnalyzer
from textblob import Blobber
tba = Blobber(analyzer=NaiveBayesAnalyzer())

In [14]:
print tba(application['question'][0]).sentiment

Sentiment(classification='neg', p_pos=0.2430232741273017, p_neg=0.7569767258726988)


In [15]:
blob.sentences 

[Sentence("I have a 9 year old Badger 1 that needs replacing, will this Badger 1 install just like the original one?")]

In [16]:
blob = TextBlob(application['question'][0], analyzer=PatternAnalyzer())
blob.sentences 

[Sentence("I have a 9 year old Badger 1 that needs replacing, will this Badger 1 install just like the original one?")]

In [17]:
#Polarity ranges from -1 to 1 (1 = positive sentiment). 
#Subjectivity how much opinion is expressed within a given sentence: 0 = objective 1 = subjective
blob.sentiment

Sentiment(polarity=0.2375, subjectivity=0.475)

#### #.

## 2.1 Analyze the questions

### 2.1.1 questionType

In [71]:
q_application = application[['questionType','asin','question']]

In [72]:
q_application.head()

,questionType,asin,question
0,yes/no,B00004U9JP,I have a 9 year old Badger 1 that needs replac...
1,open-ended,B00004U9JP,model number
2,yes/no,B00004U9JP,can I replace Badger 1 1/3 with a Badger 5 1/2...
3,yes/no,B00004U9JP,Does this come with power cord and dishwasher ...
4,open-ended,B00004U9JP,loud noise inside when turned on. sounds like ...


In [73]:
mapping = {"yes/no": 1, "open-ended": 0}
q_application['questionType'] = q_application['questionType'].replace(mapping).astype(np.int)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
q_application.head()

,questionType,asin,question
0,1,B00004U9JP,I have a 9 year old Badger 1 that needs replac...
1,0,B00004U9JP,model number
2,1,B00004U9JP,can I replace Badger 1 1/3 with a Badger 5 1/2...
3,1,B00004U9JP,Does this come with power cord and dishwasher ...
4,0,B00004U9JP,loud noise inside when turned on. sounds like ...


#### train queationType classify

In [22]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

In [23]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn.externals import joblib

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
X_train = q_application.question.values
y_train = q_application.questionType.values
print X_train.shape
print y_train.shape

(9011,)
(9011,)


In [25]:
vect = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(q_application.question.values, q_application.questionType.values, random_state=1, train_size=0.75)
train_dtm = vect.fit_transform(X_train)
print 'Features: ', train_dtm.shape[1]
test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(train_dtm, y_train)
y_pred_class = nb.predict(test_dtm)
print 'Accuracy: ', metrics.accuracy_score(y_test, y_pred_class)

Features:  7582
Accuracy:  0.7585441633377719


In [26]:
vect = CountVectorizer()
train_dtm = vect.fit_transform(X_train)
print train_dtm.shape

(6758, 7582)


In [27]:
transformer = TfidfTransformer()
train_tfidf = transformer.fit_transform(train_dtm)

In [28]:
nb = MultinomialNB().fit(train_tfidf, y_train)

In [29]:
# save the training model & vector
joblib.dump(nb, '/data/QA/model.pkl')
joblib.dump(vect, '/data/QA/count_vect')

['/data/QA/count_vect']

In [30]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()

In [31]:
from nltk.stem.snowball import SnowballStemmer
snowball_stemmer = SnowballStemmer('english')

In [32]:
lemmatizer = nltk.WordNetLemmatizer()

In [33]:
def spit_into_lemmas(text):
    text = unicode(text, 'utf-8').lower()
    words = TextBlob(text).words
    return [word.lemmatize() for word in words]

In [34]:
vect = CountVectorizer(analyzer=spit_into_lemmas)
train_dtm = vect.fit_transform(X_train)
print 'Features: ', train_dtm.shape[1]
test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(train_dtm, y_train)
y_pred_class = nb.predict(test_dtm)
print 'Accuracy: ', metrics.accuracy_score(y_test, y_pred_class)

Features:  7509
Accuracy:  0.7612072791833111


#### do a new prediction

In [35]:
clf = joblib.load('/data/QA/model.pkl')

In [36]:
count_vect = joblib.load('/data/QA/count_vect')

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(q_application.question.values, q_application.questionType.values, random_state=1, train_size=0.75)

In [38]:
#testing_data = [q_application['question'][0]]
#testing_data = ['loud noise inside when turned on. sounds like']
#testing_data = ['can I replace Badger 1 1/3 with a Badger?']
testing_data = ['can I ask a question?']



In [39]:
tfidf_transformer = TfidfTransformer()

In [40]:
X_new_counts = count_vect.transform(testing_data)

In [41]:
X_new_tfidf = tfidf_transformer.fit_transform(X_new_counts)

In [42]:
prediction = clf.predict(X_new_tfidf)
print prediction

[0]


## train question-answer model

In [66]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

In [67]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn.externals import joblib

In [70]:
q_a = application[['questionType','asin','question','answer']]


,questionType,asin,question,answer
0,yes/no,B00004U9JP,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.
1,open-ended,B00004U9JP,model number,This may help InSinkErator Model BADGER-1: Bad...
2,yes/no,B00004U9JP,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...
3,yes/no,B00004U9JP,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...
4,open-ended,B00004U9JP,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...


In [74]:
mapping = {"yes/no": 1, "open-ended": 0}
q_a['questionType'] = q_a['questionType'].replace(mapping).astype(np.int)
q_a.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,questionType,asin,question,answer
0,1,B00004U9JP,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.
1,0,B00004U9JP,model number,This may help InSinkErator Model BADGER-1: Bad...
2,1,B00004U9JP,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...
3,1,B00004U9JP,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...
4,0,B00004U9JP,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...


In [76]:
X_train = q_a.question.values
y_train = q_a.answer.values
print X_train.shape
print y_train.shape

(9011,)
(9011,)


In [80]:
vect = CountVectorizer(ngram_range=(2,2),stop_words='english', max_features=10000)
X_train, X_test, y_train, y_test = train_test_split(q_a.question.values, q_a.answer.values, random_state=1, train_size=0.75)
train_dtm = vect.fit_transform(X_train)
print 'Features: ', train_dtm.shape[1]
test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(train_dtm, y_train)
y_pred_class = nb.predict(test_dtm)
print 'Accuracy: ', metrics.accuracy_score(y_test, y_pred_class)

Features:  10000
Accuracy:  0.029738126941855306


### 2.1.2 question similarity

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
test = raw_input("Hi, how can I help you?\n")

In [57]:
test

'how to use a washer?'

In [58]:
data = q_application.question.values
data=np.append(data,test)

In [59]:
vec = TfidfVectorizer()
X = vec.fit_transform(data)

In [60]:
S = cosine_similarity(X)

In [61]:
def select_k_closest(target, k, distance_matrix, data):
   
    res = list(np.argsort(distance_matrix[target])[-k-1:-1])
    res.reverse()   
    result = [ (S[target,i], data[i]) for i in res]
    
    return result

In [62]:
k = 4
target = len(data)-1
question = select_k_closest(target, k, S, data)
question

[(0.4531438669924055, 'how loud is this washer?'),
 (0.4530932875169346, 'how to remove front of washer to get to pump'),
 (0.44047534165057606, 'how much detergent do I use in the Danby 1.7 washer?'),
 (0.3896558331924539, 'How do you use the disposal')]

In [63]:
def getAnswer(question):    ##question is the result of select_k_closest
    answer_set = []
    for i in range(len(question)):
        answer_set.append(application[application['question']==question[i][1]])
    return answer_set[0]['answer']   ##return the first answer
#    return answer_set

In [64]:
getAnswer(question)

7935    compared to maytag I can barely tell the dishe...
Name: answer, dtype: object

In [83]:
import nltk